In [ ]:
import zipfile
import os

# ZIP dosyanın Colab içindeki yolu (Dosyanı yüklediğin yer burası olmalı)
zip_path = "/content/drive/MyDrive/Beyin_Verileri_Teknofest_2021/beyin_verileri.zip"

# ZIP dosyasını çıkaracağımız hedef klasör
extract_path = "/content/beyin_verileri"

# ZIP dosyasını çıkar
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)  # Kendi klasörünü oluştur

print("ZIP dosyası çıkarıldı.")

# Çıkarılan klasör içeriğini kontrol et
print(os.listdir(extract_path))
print("Train klasörü:", os.listdir(os.path.join(extract_path, "train")))
print("Val klasörü:", os.listdir(os.path.join(extract_path, "val")))
print("Test klasörü:", os.listdir(os.path.join(extract_path, "test")))

ZIP dosyası çıkarıldı.
['val', 'train', 'test']
Train klasörü: ['inme_yok', 'inme_var']
Val klasörü: ['inme_yok', 'inme_var']
Test klasörü: ['inme_yok', 'inme_var']


In [ ]:
import numpy as np
import cv2
import os

# Görüntü boyutu
img_size = (128, 128)

# Genel veri yükleme fonksiyonu (ZIP'ten çıkarılan veriler için)
def load_dataset(dataset_type="train", limit=None):
    X, Y = [], []
    classes = ["inme_var", "inme_yok"]

    # ZIP'ten çıkarılan verinin yolu
    dataset_path = f"/content/beyin_verileri/{dataset_type}"

    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Dataset path '{dataset_path}' not found!")

    for label, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)

        if not os.path.exists(class_path):
            print(f"Warning: Class path '{class_path}' not found! Skipping...")
            continue

        image_files = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.endswith('.png')]

        if not image_files:
            print(f"Warning: No images found in {class_path}")

        # Eğer limit varsa, sadece belirli sayıda görüntü al
        if limit:
            image_files = image_files[:limit]

        for file in image_files:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # Gri tonlamalı oku

            if img is None:
                print(f"Warning: Could not read image {file}. Skipping...")
                continue

            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalizasyon (0-1 arasına getir)
            img = np.expand_dims(img, axis=-1)  # CNN için uygun hale getir

            X.append(img)
            Y.append(label)

    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.int32)

# Train, Validation ve Test veri setlerini yükleme
X_train, Y_train = load_dataset("train")
X_val, Y_val = load_dataset("val")
X_test, Y_test = load_dataset("test")

# Veri boyutlarını kontrol et
print(f"Train Set: {X_train.shape}, Labels: {Y_train.shape}")
print(f"Validation Set: {X_val.shape}, Labels: {Y_val.shape}")
print(f"Test Set: {X_test.shape}, Labels: {Y_test.shape}")


Train Set: (4654, 128, 128, 1), Labels: (4654,)
Validation Set: (997, 128, 128, 1), Labels: (997,)
Test Set: (999, 128, 128, 1), Labels: (999,)


In [ ]:
# EfficientNet için görüntüleri 3 kanallı hale getirme fonksiyonu
def convert_to_rgb(X):
    return np.repeat(X, 3, axis=-1)  # Gri kanalın 3 kopyasını oluştur

# Veriyi 3 kanallı hale getir
X_train = convert_to_rgb(X_train)
X_val = convert_to_rgb(X_val)
X_test = convert_to_rgb(X_test)

print(f"Train Set (RGB): {X_train.shape}")
print(f"Validation Set (RGB): {X_val.shape}")
print(f"Test Set (RGB): {X_test.shape}")


Train Set (RGB): (4654, 128, 128, 3)
Validation Set (RGB): (997, 128, 128, 3)
Test Set (RGB): (999, 128, 128, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# EfficientNetB3 Modelini Yükle (128x128 giriş boyutu ile)
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Önceki katmanları dondur (pretrained ağırlıkları korumak için)
base_model.trainable = False

# Özellikleri Çıkar
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Flatten yerine, ortalama havuzlama
x = BatchNormalization()(x)  # Normalizasyon
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)  # Daha geniş bir Dense katmanı
x = Dropout(0.5)(x)  # Aşırı öğrenmeyi önlemek için Dropout
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # Daha fazla kapasite
x = Dropout(0.4)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)  # Çıkış katmanını x'e bağladık

# Modeli oluştur
model = Model(inputs=base_model.input, outputs=x)

# Modeli derle
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Optimize Edilmiş Öğrenme Oranı
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

# Modeli göster
model.summary()


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 128, 128, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 128, 128, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 128, 128, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 129, 129, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 64, 64, 40)     │          1,080 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 64, 64, 40)     │            160 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 64, 64, 40)     │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 64, 64, 40)     │            360 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 64, 64, 40)     │            160 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 64, 64, 40)     │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 40)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 40)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 10)       │            410 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 11,740,976 (44.79 MB)

 Trainable params: 954,369 (3.64 MB)

 Non-trainable params: 10,786,607 (41.15 MB)

In [ ]:
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_val shape: {Y_val.shape}")


Y_train shape: (4654,)
Y_val shape: (997,)


In [ ]:
# Y_train, Y_val ve Y_test'in boyutlarını düzeltme
Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)
Y_test = Y_test.reshape(-1, 1)

# Yeni boyutları kontrol et
print(f"Yeni Y_train shape: {Y_train.shape}")
print(f"Yeni Y_val shape: {Y_val.shape}")
print(f"Yeni Y_test shape: {Y_test.shape}")


Yeni Y_train shape: (4654, 1)
Yeni Y_val shape: (997, 1)
Yeni Y_test shape: (999, 1)


In [ ]:
import os
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Modelin kaydedileceği Google Drive yolu
model_save_path = "/content/drive/MyDrive/Model_Kayitlar/EfficientNetB3"

class EpochCheckpoint(Callback):
    def __init__(self, save_path, save_freq=25):
        super(EpochCheckpoint, self).__init__()
        self.save_path = save_path
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:  # Epoch 25, 50, 75, ... olduğunda kaydet
            filepath = os.path.join(self.save_path, f"EfficientNetB3_Epoch_{epoch+1:03d}.keras")
            self.model.save(filepath)
            print(f"\nEpoch {epoch+1}: Model saved to {filepath}")

# Callback listesi
callbacks = [
    # Overfitting'i önlemek için EarlyStopping
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        verbose=1,
        mode='min',
        restore_best_weights=True
    ),

    # Öğrenme oranını düşürmek için ReduceLROnPlateau
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),

    # En iyi modeli kaydetmek için ModelCheckpoint (En iyi model yalnızca kaydedilecek)
    ModelCheckpoint(
        filepath=os.path.join(model_save_path, "Best_Model_EfficientNetB3.keras"),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        save_weights_only=False,
        verbose=2
    ),

    # Her 10 epoch'ta bir kaydetmek için EpochCheckpoint
    EpochCheckpoint(
        save_path=model_save_path,
        save_freq=10  # Her 10 epoch'ta kaydetsin
    )
]


In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=1000,  # Maksimum epoch sayısı
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/1000
 43/146 ━━━━━━━━━━━━━━━━━━━━ 2:02 1s/step - accuracy: 0.5775 - loss: 2.0330 - precision: 0.6797 - recall: 0.6917

In [ ]:
import matplotlib.pyplot as plt

# Eğitim ve doğrulama kayıpları
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.show()

# Eğitim ve doğrulama doğruluğu
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Over Epochs')
plt.show()

# Precision ve Recall
plt.figure(figsize=(12, 5))
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.title('Precision Over Epochs')
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.title('Recall Over Epochs')
plt.show()

In [ ]:
# Test veri setinde modeli değerlendir
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, Y_test)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
